In [16]:
# Script to create date for Date Dimesion Staging
# Import required libraries
import sys
from lib.spark_session import get_spark_session
from lib.utils import date_data, get_string_cols, get_rundate
from lib.job_control import insert_log, get_max_timestamp
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import current_timestamp, expr, to_date, lit,to_timestamp,lit,coalesce,split
from datetime import datetime
from delta import DeltaTable

In [2]:
# JOB Parameters
rundate = get_rundate()
schema_name = "edw_stg"
table_name = "dim_customer_stg"
table_full_name = f"{schema_name}.{table_name}"
landing_table_full_name = "edw_ld.dim_customer_ld"
print("SPARK_APP: JOB triggered for rundate - " + rundate)

SPARK_APP: JOB triggered for rundate - 20220101


In [3]:
spark:SparkSession = get_spark_session(f"staging_load: {table_full_name}")
print("SPARK_APP: Spark UI - " + spark.sparkContext.uiWebUrl)

SPARK_APP: Spark UI - http://03205cdd01e3:4040


In [4]:
spark.conf.set("spark.sql.shuffle.partitions",8)
spark.conf.set("spark.sql.parquet.mergeSchema",True)

In [6]:
# Get the max_timestamp for data load in staging
max_timestamp = get_max_timestamp(spark, schema_name, table_name)
print("SPARK_APP: Max timestamp for staging data load - " + str(max_timestamp))

SPARK_APP: Max timestamp for staging data load - 1900-01-01 00:00:00.000000


In [7]:
df_ld = spark \
    .read \
    .table(landing_table_full_name) \
    .where(f"insert_dt > to_timestamp('{max_timestamp}')")

print("SPARK_APP: Landing Data Count - " + str(df_ld.count()))
print("SPARK_APP: Printing Landing Schema --")
df_ld.printSchema()

SPARK_APP: Landing Data Count - 18
SPARK_APP: Printing Landing Schema --
root
 |-- customer_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- email: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- plan_type: string (nullable = true)
 |-- insert_dt: timestamp (nullable = true)
 |-- rundate: string (nullable = true)



In [8]:
# De-dupe the data based on NK
df_dedupe = df_ld.withColumn("_rnk", expr(f"row_number() over (partition by customer_id order by insert_dt desc)")) \
    .where("_rnk = 1").drop("_rnk")

print("SPARK_APP: Landing Data Count after de-dupe - " + str(df_dedupe.count()))

SPARK_APP: Landing Data Count after de-dupe - 18


In [9]:
df_dedupe.show(5,truncate = False)

+-----------+-----------------+--------------+-----------+-----+--------+--------------+--------------------------+-------------+---------+--------------------------+--------+
|customer_id|name             |address       |city       |state|zip_code|phone_number  |email                     |date_of_birth|plan_type|insert_dt                 |rundate |
+-----------+-----------------+--------------+-----------+-----+--------+--------------+--------------------------+-------------+---------+--------------------------+--------+
|C001       |Ramesh Kumar     |123 Main St   |Anytown    |WB   |12345   |91-00000-00000|ramesh@email.com          |01-01-1980   |P        |2024-05-30 02:46:24.260469|20220101|
|C002       |Meena            |456 Elm St    |Anothertown|GJ   |67890   |91-00000-00001|meena@email.com           |02-02-1985   |G        |2024-05-30 02:46:24.260469|20220101|
|C003       |Imitiaz Ali      |789 Oak Ave   |Bigcity    |JK   |9876    |91-00000-00002|imtiaz@email.com          |03-03

In [18]:
#creating temp df to make new columns
df_temp = df_dedupe.withColumn("first_name",split("name"," ")[0]).\
withColumn("last_name",split("name"," ")[1]).\
withColumn("date_of_birth", to_date("date_of_birth","dd-mm-yyyy")).\
withColumn("plan_type",coalesce("plan_type","plan_type",lit("NA"))).\
withColumn("effective_start_date",current_timestamp()).\
withColumn("effective_end_date",to_timestamp(lit("9999-12-31 00:00:00.000000"))).\
withColumn("active_flag",lit(1)).\
withColumn("insert_dt",current_timestamp()).\
withColumn("update_dt", current_timestamp())

In [19]:
df_temp.show(5,False)

+-----------+-----------------+--------------+-----------+-----+--------+--------------+--------------------------+-------------+---------+--------------------------+--------+----------+---------+--------------------------+-------------------+-----------+--------------------------+
|customer_id|name             |address       |city       |state|zip_code|phone_number  |email                     |date_of_birth|plan_type|insert_dt                 |rundate |first_name|last_name|effective_start_date      |effective_end_date |active_flag|update_dt                 |
+-----------+-----------------+--------------+-----------+-----+--------+--------------+--------------------------+-------------+---------+--------------------------+--------+----------+---------+--------------------------+-------------------+-----------+--------------------------+
|C001       |Ramesh Kumar     |123 Main St   |Anytown    |WB   |12345   |91-00000-00000|ramesh@email.com          |1980-01-01   |P        |2024-05-31 0

In [20]:
df_temp.write.format("delta").mode("overwrite").saveAsTable(table_full_name)

print("SPARK_APP: Data written to staging table")

SPARK_APP: Data written to staging table


In [21]:
# Add job details in JOB CONTROL
insert_log(spark, schema_name, table_name, datetime.now(), rundate)
print("SPARK_APP: Update JOB Control Log")

SPARK_APP: Update JOB Control Log


In [22]:
spark.sql(f"select * from edw.job_control where table_name = '{table_name}' order by insert_dt desc limit 1").show(truncate=False)

+-----------+----------------+--------------------------+--------+--------------------------+
|schema_name|table_name      |max_timestamp             |rundate |insert_dt                 |
+-----------+----------------+--------------------------+--------+--------------------------+
|edw_stg    |dim_customer_stg|2024-05-31 03:34:56.952844|20220101|2024-05-31 03:34:57.922558|
+-----------+----------------+--------------------------+--------+--------------------------+



In [23]:
# Get the logs from delta table version
dt = DeltaTable.forName(spark, table_full_name)
dt.history().limit(1).select("version","operationMetrics.executionTimeMs", 
                                 "operationMetrics.numTargetRowsInserted",
                                "operationMetrics.numTargetRowsUpdated",
                                "operationMetrics.numOutputRows").show(1, False)

+-------+---------------+---------------------+--------------------+-------------+
|version|executionTimeMs|numTargetRowsInserted|numTargetRowsUpdated|numOutputRows|
+-------+---------------+---------------------+--------------------+-------------+
|0      |null           |null                 |null                |18           |
+-------+---------------+---------------------+--------------------+-------------+



In [24]:
# Generate Symlink manifest for Athena Access
dt.generate("symlink_format_manifest")
print("SPARK_APP: Symlink Manifest file generated")

SPARK_APP: Symlink Manifest file generated


In [26]:
spark.sql(f"select * from edw_stg.dim_customer_stg limit 10").show(truncate=False)

+-----------+-----------------+--------------+-----------+-----+--------+--------------+--------------------------+-------------+---------+--------------------------+--------+----------+---------+--------------------------+-------------------+-----------+--------------------------+
|customer_id|name             |address       |city       |state|zip_code|phone_number  |email                     |date_of_birth|plan_type|insert_dt                 |rundate |first_name|last_name|effective_start_date      |effective_end_date |active_flag|update_dt                 |
+-----------+-----------------+--------------+-----------+-----+--------+--------------+--------------------------+-------------+---------+--------------------------+--------+----------+---------+--------------------------+-------------------+-----------+--------------------------+
|C001       |Ramesh Kumar     |123 Main St   |Anytown    |WB   |12345   |91-00000-00000|ramesh@email.com          |1980-01-01   |P        |2024-05-31 0

In [27]:
spark.stop()